# Title 1 Enrollment Across the Austin Area

## Hypothesis	
If Austin ISD school enrollment decreases, then the proportion of title 1 students to non title 1 students will increase

## Null	
If Austin ISD school enrollment decreases, then the proportion of title 1 students to non title 1 students will decrease or not change		

![title](AISD.jpg)

# Our Data Sources:

1. CSV files from the TEA(Texas Education Agency) website.
2. GreatSchools.org API
3. census API
4. google maps API


In [4]:
# Dependencies
from keys import googleKey
import gmaps
import requests
import time
from IPython.display import display
import ipywidgets as widgets
import SchoolExplorer as se

ModuleNotFoundError: No module named 'keys'

In [6]:
from plotly import __version__
import cufflinks as cf
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import numpy as np
import pandas as pd

In [7]:
AustinZipCo_df= pd.read_csv("csv_files/AustinPopulation.csv")
AustinDist_df = pd.read_csv("csv_files/combined.csv")

In [28]:
c_df = pd.read_csv("csv_files/compileddata.csv")
c_df = c_df.rename(columns={'Unnamed: 0': 'Index'})

# What is Title 1?
Title I, Part A (Title I) of the Elementary and Secondary Education Act, as amended by the Every Student Succeeds Act (ESEA) provides financial assistance to local educational agencies (LEAs) and schools with high numbers or high percentages of children from low-income families to help ensure that all children meet challenging state academic standards.

# The Austin Population Heat Map

In [9]:

# Store 'Lat' and 'Lng' into  locations 
locations = AustinZipCo_df[["LAT", "LNG"]].astype(float)

# Convert Population to float and store
# HINT: be sure to handle NaN values
population = AustinZipCo_df["Percent Population"].astype(float)
fig4 = gmaps.figure(layout={'width': '1000px','height': '1000px','border': '1px solid black','padding': '1px'})
population_layer = gmaps.heatmap_layer(locations, weights=population, 
                                 dissipating=False, max_intensity=6,
                                 point_radius = 0.05)

fig4.add_layer(population_layer)

fig4

Figure(layout=FigureLayout(border='1px solid black', height='1000px', padding='1px', width='1000px'))

In [5]:
df_full = pd.read_csv('csv_files/AustinSchools.csv')
gmaps.configure(api_key=googleKey.gkey)
df = df_full[['schoolName', 'latitude', 'longitude','enrollment','type']]
df = df.rename(columns={"type":"schoolType"})
df["Percent enrollment"] = 100 * df["enrollment"] / df["enrollment"].sum()
df.fillna(0, inplace=True)
se.SchoolExplorer(df).render()

In [8]:
df = AustinDist_df[AustinDist_df['DISTRICT NAME'].notnull()].reset_index(drop=True)

,Unnamed: 0,AGG_LEVEL,CHARTER STATUS,DISTRICT NAME,DISTRICT NUMBER,REGION,SCOPE,TOTAL AT RISK STUDENTS,TOTAL BILINGUAL STUDENTS,TOTAL CTE STUDENTS,...,TOTAL FOSTER CARE STUDENTS,TOTAL G & T STUDENTS,TOTAL HOMELESS STUDENTS,TOTAL LEP STUDENTS,TOTAL MILITARY CONNECTED STUDENTS,TOTAL PRE-K FOSTER STUDENTS,TOTAL PRE-K MILITARY STUDENTS,TOTAL STUDENTS RECEIVING SPECIAL EDUCATION SERVICES,TOTAL TITLE I STUDENTS,YEAR
0,0,DISTRICT,NaN,CAYUGA ISD,1902.0,7.0,STATE,224.0,0.0,153.0,...,-9999999.0,63.0,NaN,-9999999.0,5.0,NaN,NaN,80.0,570.0,2015-2016
1,1,DISTRICT,NaN,ELKHART ISD,1903.0,7.0,STATE,380.0,0.0,304.0,...,14.0,45.0,NaN,19.0,0.0,NaN,NaN,144.0,1244.0,2015-2016
2,2,DISTRICT,NaN,FRANKSTON ISD,1904.0,7.0,STATE,338.0,0.0,273.0,...,11.0,60.0,NaN,19.0,16.0,NaN,NaN,70.0,841.0,2015-2016


In [9]:
#Obtains the average of masked values based on 1 percent of total enrollment
t1_missing_val = df[df['TOTAL TITLE I STUDENTS']<0]['TOTAL ENROLLMENT'].mean()*0.01
#This grabs the working dataframe columns from the orignal df, and will replace them with the t1_missing_val mean
wk_df = df[['YEAR','DISTRICT NAME','DISTRICT NUMBER','REGION','TOTAL ENROLLMENT','TOTAL TITLE I STUDENTS']]
wk_df.replace(-9999999,t1_missing_val)
wk_df['PCT_T1'] = (wk_df['TOTAL TITLE I STUDENTS'] / wk_df['TOTAL ENROLLMENT']) * 100
wk_df = wk_df.round()

In [10]:
def yr_enrollment(limit, year='2018-2019'):
    '''Sort district ISDs by number of enrollment and academic year.
    limit: takes in an integer or float
    year: takes in a string of academic year'''
    try:
        data = wk_df[(wk_df['TOTAL ENROLLMENT'] >= limit) & 
        (wk_df['YEAR'] == year)][['DISTRICT NUMBER','DISTRICT NAME','TOTAL ENROLLMENT','TOTAL TITLE I STUDENTS']]
        data = data.sort_values('TOTAL ENROLLMENT', ascending=False).reset_index(drop=True)
        data['PCT_T1'] = (data['TOTAL TITLE I STUDENTS'] / data['TOTAL ENROLLMENT']) * 100
        return data.round()
    except Exception as e:
        return print('Please read the docsting notes')

In [11]:
tx_enroll = df[df['YEAR']=='2018-2019']['TOTAL ENROLLMENT'].sum()
tx_t1enroll = df[df['YEAR']=='2018-2019']['TOTAL TITLE I STUDENTS'].sum()
pct_t1 = ((tx_t1enroll/tx_enroll) * 100).round()
print(f'Texas total current enrollment: {tx_enroll}.')
print(f'Texas total current title 1 enrollment: {tx_t1enroll}.')
print(f'Texas total title 1 percent enrollment: {pct_t1} percent.')

In [13]:
yr_enrollment(65000)

,DISTRICT NUMBER,DISTRICT NAME,TOTAL ENROLLMENT,TOTAL TITLE I STUDENTS,PCT_T1
0,101912.0,HOUSTON ISD,209772.0,191412.0,91.0
1,57905.0,DALLAS ISD,155119.0,150628.0,97.0
2,101907.0,CYPRESS-FAIRBANKS ISD,116512.0,51658.0,44.0
3,15915.0,NORTHSIDE ISD,106501.0,37151.0,35.0
4,220905.0,FORT WORTH ISD,84510.0,81088.0,96.0
5,227901.0,AUSTIN ISD,80032.0,40203.0,50.0
6,101914.0,KATY ISD,79913.0,14254.0,18.0
7,79907.0,FORT BEND ISD,76122.0,15198.0,20.0
8,101902.0,ALDINE ISD,66854.0,66714.0,100.0
9,15910.0,NORTH EAST ISD,65186.0,15908.0,24.0


In [14]:
#names of all ISD's surrounding Austin area.
aus_area_schools = ['AUSTIN ISD','DEL VALLE ISD','DRIPPING SPRINGS ISD',
 'EANES ISD','GEORGETOWN ISD','HAYS CISD','HUTTO ISD','LAKE TRAVIS ISD',
 'LEANDER ISD','PFLUGERVILLE ISD','ROUND ROCK ISD','MANOR ISD','LAGO VISTA ISD']

In [22]:
aus_area_df = wk_df[wk_df['DISTRICT NAME'].isin(aus_area_schools)].reset_index(drop=True)

In [16]:
wk_plot = aus_area_df.groupby(["DISTRICT NAME", "YEAR"])['TOTAL ENROLLMENT'].mean().reset_index()

In [17]:
#Plot enrollments of all Austin area I
wk_pivot = wk_plot.pivot(index='DISTRICT NAME', columns='YEAR')
cf.set_config_file(offline=True, world_readable=False, theme='ggplot')
wk_pivot.iplot(kind='bar',title='Autin Area ISDs')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [18]:
t1_plot = aus_area_df.groupby(["DISTRICT NAME", "YEAR"])['TOTAL TITLE I STUDENTS'].mean().reset_index()

In [19]:
t1_pivot = t1_plot.pivot(index='DISTRICT NAME', columns='YEAR')
cf.set_config_file(offline=True, world_readable=False, theme='ggplot')
t1_pivot.iplot(kind='bar',title='Autin Area ISDs Title 1 Status')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [20]:
t1_pct_plot = aus_area_df.groupby(["DISTRICT NAME", "YEAR"])['PCT_T1'].mean().reset_index()

In [21]:
t1_pct_pivot = t1_pct_plot.pivot(index='DISTRICT NAME', columns='YEAR')
cf.set_config_file(offline=True, world_readable=False, theme='ggplot')
t1_pct_pivot.iplot(kind='bar',title='Autin Area ISDs Title 1 Pecent Status')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [33]:
# title one vs. non-title one
# Calculate proportion of title one to non-title one students
c_df['non_title1'] = c_df['total_enroll']- c_df['total_title1']
non_t1 = c_df[['year', 'total_title1', 'non_title1']]
non_t1_sum = non_t1.groupby('year').sum()

non_df = non_t1_sum.reset_index()
non_df['Percentage'] = (non_df['non_title1']/non_df['total_title1'])*100
non_df['Ratio'] = (100/non_df['Percentage'])
non_df['T1_rate_of_change'] = (non_df.iloc[0,1]-non_df['total_title1'])*(-1)
non_df['n-T1_rate_of_change'] = (non_df.iloc[0,2]-non_df['non_title1'])*(-1)
non_df['n-T1_negatvie_rate_of_change'] = (non_df.iloc[0,2]-non_df['non_title1'])
rate_enr_t1 = non_df[['year', 'T1_rate_of_change', 'n-T1_negatvie_rate_of_change']]
non_df

df_nnn = non_df[['year','Ratio']].set_index('year')
df_nnn.iplot(kind='line', yTitle='Ratio Title One to Non-Title One', xTitle='Years', title="Title One to Non-Title One Ratio")

In [29]:
# group by data
# year
year_df = c_df.groupby('year').sum()
# year and school type
year_type_df = c_df.groupby(['school_type', 'year']).sum()
year_df
# Caclulate and pull out column for percentage of title one students per year
year_type_df['T1_rate'] = (year_type_df['total_title1']/year_type_df['total_enroll'])*100
year_type_df
df_s = year_type_df['T1_rate']
df_s.head()

df_s.unstack().iplot(kind='bar', yTitle='Title One Student Enrollment (%)', xTitle='School Type', title="Title One Student Percentage by School Type")

In [30]:
# find the top 5 populate high schools
high_ind = c_df.set_index('school_type')
high = high_ind.loc['High'].groupby(['year', 'campus']).sum()
labels=["Bowie High School","Akins High School","Anderson High School","Austin High School","Mccallum High School"]
high_t1 = pd.DataFrame(high[['total_title1', 'total_enroll']]).reset_index()
high_t1['title1_%'] = (high_t1['total_title1']/high_t1['total_enroll'])*100
top_high = high_t1.set_index('campus')
top_high = top_high[['title1_%', 'year']]
top_5 = top_high.loc[["Bowie High School","Akins High School","Anderson High School","Austin High School","Mccallum High School"]]
top_5 = top_5.groupby(['campus', 'year']).sum()
top_5.head()
top_5.unstack().iplot(kind='bar', yTitle='Student Enrollment', xTitle='Campus Name', title="Student Enrollment Overview")